<a href="https://colab.research.google.com/github/ViekyKayal/AlcoAudio/blob/master/semantic%20similarities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search Demo Based on UAE

In [ ]:
!pip install angle_emb faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 28.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from tqdm import tqdm


ds = load_dataset('nlphuji/flickr30k')
docs = []
for obj in tqdm(ds['test']):
    docs += obj['caption']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|██████████| 31014/31014 [01:38<00:00, 315.34it/s]


In [ ]:
from angle_emb import AnglE

uae = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
# index

import numpy as np
import faiss                   # make faiss available
np.random.seed(1234)           # make reproducible


d = 1024                       # dimension
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)

True


In [ ]:
from boltons.iterutils import chunked


chunk_size = 64

for chunk in tqdm(chunked(docs, chunk_size)):
    for vec in uae.encode(chunk):
        index.add(vec.reshape(-1, d))


100%|██████████| 2423/2423 [18:01<00:00,  2.24it/s]


In [ ]:
query = 'A man riding a small boat in a harbor.'
vec = uae.encode(query)

D, I = index.search(vec, 5)
print(f'query: {query}')
print('Top 5 related docs:')
for docid in I[0]:
    print(docs[docid])


query: A man riding a small boat in a harbor.
Top 5 related docs:
A man riding a small boat in a harbor, with fog and mountains in the background.
A man is riding in a boat on the water.
A man is on a small boat with his hand on the boat's motor.
A man in a blue boat on the water.
A man maneuvers a thatched covered boat through the turquoise colored water.


In [ ]:
query = 'A dog runs on the green grass near a wooden fence.'
vec = uae.encode(query)

D, I = index.search(vec, 5)
print(f'query: {query}')
print('Top 5 related docs:')
for docid in I[0]:
    print(docs[docid])


query: A dog runs on the green grass near a wooden fence.
Top 5 related docs:
A dog runs on the green grass near a wooden fence.
The dog by the fence is running on the grass.
Dog running through grass in fenced in area.
A dog runs on the green grass near a grove of trees.
A brown dog runs in the grass with a wooden fence behind.


In [ ]:
query = "A man with a goatee in a black shirt and white latex gloves is using a tattoo gun to place a tattoo on someone's back."
vec = uae.encode(query)

D, I = index.search(vec, 5)
print(f'query: {query}')
print('Top 5 related docs:')
for docid in I[0]:
    print(docs[docid])


query: A man with a goatee in a black shirt and white latex gloves is using a tattoo gun to place a tattoo on someone's back.
Top 5 related docs:
A man with a goatee in a black shirt and white latex gloves is using a tattoo gun to place a tattoo on someone's back.
Tattooed man getting a tattoo.
A man is putting a tattoo on a another's man upper back.
A man works very hard tattooing another person.
A man getting a tattoo on his back.


In [ ]:
query = "Boy in brown shirt with headphones on sits on woman's shoulders in a crowd."
vec = uae.encode(query)

D, I = index.search(vec, 5)
print(f'query: {query}')
print('Top 5 related docs:')
for docid in I[0]:
    print(docs[docid])


query: Boy in brown shirt with headphones on sits on woman's shoulders in a crowd.
Top 5 related docs:
Boy in brown shirt with headphones on sits on woman's shoulders in a crowd.
A boy wearing headphones sits on a woman's shoulders.
A kid wearing headphones sits on his mom's shoulders while in a crowd.
A boy with headphones on sitting on top of a woman's shoulders.
A boy in a red shirt is sitting on the shoulders of a man in a crowd.


In [ ]:
query = "Three dogs are standing in the grass and a person is sitting next to them"
vec = uae.encode(query)

D, I = index.search(vec, 5)
print(f'query: {query}')
print('Top 5 related docs:')
for docid in I[0]:
    print(docs[docid])


query: Three dogs are standing in the grass and a person is sitting next to them
Top 5 related docs:
Three dogs are standing in the grass and a person is sitting next to them
Three dogs stand in a grassy field while a person kneels nearby.
The three dogs are standing in tall grass.
Three multicolored dogs standing in a grassy field
Five people stand in the grass with three dogs.
